# Convert analytic satellite images to 'visible' format
- notebook code shall eventually be made a .py file
- very very preliminary stage

In [ ]:
# this is the preliminary definition of the function
def convert_sat_img(fn, src_type="BGRN", dest_type=["RGB"], do_visualize=True):
    """
    Converts analytic image (3 or 4 bands) to visual format and saves it
    fn: string, name of file
    src_type: string, bands in source file (pay attention to order)
    dest_type: string or list, output format (RGB, PCA or PCA-IR)
    do_visualize: boolean, if True, images will be displayed
    """
    pass

In [ ]:
# define a few constants = input args above
#fn = "/media/hh/hd_internal/_data_DS/DSR/satelliteImages/Borneo/3093/20170710_023840_0c75_3B_AnalyticMS.tif"
fn = "satellite_images/20180203_020044_1034_3B_AnalyticMS_SR.tif"

# src_type must be a string, any of  RGB, RGBA, BGRN
# - RGB: normal red green blue image
# - RGBA: RGB with alpha map
# - BGRN: or 4-band image with N=near infrared
src_type = "BGRN"
# dest_type must be either a string or a list of strings: any combination of RGB, PCA, and PCA-IR
# - RGB: normal red green blue image
# - PCA: grayscale image of 1st principal component
# - PCA-IR: first two principal components of RGB bands plus IR channel, so this would be
# some kind of false-color image (highly experimental)
dest_type = ["RGB","PCA"] 
do_visualize = True

In [ ]:
# check whether all are needed
import numpy as np
import rasterio 
from rasterio.plot import show
from osgeo import gdal
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,12)

#import hh_utilities as hu

In [ ]:
# input error checking:
assert(src_type in ("RGB", "RGBA", "BGRN"))
assert(not set(dest_type).difference(set(["RGB", "PCA", "PCA-IR"])))


## opening file & retrieving information

In [ ]:
# open dataset
src = rasterio.open(fn)

# 
src.colorinterp

In [ ]:
# bands and dimensions
src.count, src.width, src.height

# number type in each band
src.indexes, src.dtypes

In [ ]:
# playing around:
d = src.read(4)
## read all data into numpy array (the bands are in the first dimension!)
#d = src.read()

In [ ]:
fig, ax = plt.subplots()
show(d, ax=ax, cmap='gray')


In [ ]:
# taken from the training image generation notebook, has to be adapted
def write_tile(src, window, meta, ax_tile):
    outpath = output_tile_filename.format(TRAINING_SAT_DIR,int(window.col_off),int(window.row_off))
    with rio.open(outpath, 'w', **meta) as outds:
        tile = src.read(window=window)
        rioplot.show(tile, ax=ax_tile, cmap='gray')
        outds.write(tile)